## Topic modeling and word2vec

This notebook examines two topic modeling models from the `gensim` library:
  - LDA (Latent Dirichlet Allocation)
  - word2vec

Sources of inspiration:
  - https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html
  - https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

### LDA (Latent Dirichlet Allocation)
We install the topic modeling library gensim (http://radimrehurek.com/gensim/) and load the NLTK library (http://nltk.org/), which will be needed for lemmatization.


In [1]:
!pip install --upgrade gensim
!pip install --upgrade nltk

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/2a/15/aca2fc3b9e97bd0e28be4a4302793c43757b04b828223c6d103c72132f19/gensim-4.3.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for scipy<1.14.0,>=1.7.0 from https://files.pythonhosted.org/packages/ba/92/42476de1af309c27710004f5cdebc27bec62c204db42e05b23a302cb0c9a/scipy-1.13.1-cp311-cp311-macosx_12_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 709.0 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/65/12/cc24847b4b0b124501a33cd8f7963f79f6f6584bc7f2f4fc16bbbaa54c8f/smart_open-7.0.4-py3-none-any.whl.metadata
  Using cached smart_open-7.0.4-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for wrapt from https://files.pythonhosted.org/packages/0f/16/ea627d7817394db04518f62934a5de59874b587b792300991b3c347ff5e0/wrapt-1.16.0-cp311-cp311-macosx_11

In [2]:
import nltk
import numpy as np
import os
import sys

from gensim import corpora, models, similarities
from math import log
from time import time

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

We read the collection of source texts into a list of documents. Each document is a list of lemmas (tokens). In this example, we load the entire collection into memory. In fact, `gensim` allows you to avoid this at all stages of model building.

The collection used is articles from the NeurIPS conference, one of the standard collections for topic modeling. The number of documents is about 1700, with each document having a length of 1000-2000 words. 

In [3]:
import tarfile
import re
import urllib.request, zipfile


tarfile_url = 'https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'
filename = 'nips12raw_str602.tgz'
urllib.request.urlretrieve(tarfile_url, filename)

def extract_documents(fname=filename):
    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')


In [4]:
docs = list(extract_documents())
print(len(docs))
print(print(docs[0][:500]))

1740
1 
CONNECTIVITY VERSUS ENTROPY 
Yaser S. Abu-Mostafa 
California Institute of Technology 
Pasadena, CA 91125 
ABSTRACT 
How does the connectivity of a neural network (number of synapses per 
neuron) relate to the complexity of the problems it can handle (measured by 
the entropy)? Switching theory would suggest no relation at all, since all Boolean 
functions can be implemented using a circuit with very low connectivity (e.g., 
using two-input NAND gates). However, for a network that learns a pr
None


Data preparation:
- Create a dictionary
- Perform lemmatization
- Build n-grams
- Filter out tokens that are too frequent or too rare

In [5]:
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

In [6]:
print(np.sum([len(doc) for doc in docs]))

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

print(np.sum([len(doc) for doc in docs]))

5461201
5115888


In [7]:
print(docs[1][:50])

['stochastic', 'learning', 'networks', 'and', 'their', 'electronic', 'implementation', 'joshua', 'alspector', 'robert', 'b', 'allen', 'victor', 'hut', 'and', 'srinagesh', 'satyanarayana', 'bell', 'communications', 'research', 'morristown', 'nj', 'abstract', 'we', 'describe', 'a', 'family', 'of', 'learning', 'algorithms', 'that', 'operate', 'on', 'a', 'recurrent', 'symmetrically', 'connected', 'neuromorphic', 'network', 'that', 'like', 'the', 'boltzmann', 'machine', 'settles', 'in', 'the', 'presence', 'of', 'noise']


In [8]:
# Remove words that are only one character
docs = [[token for token in doc if len(token) > 1] for doc in docs]
print(np.sum([len(doc) for doc in docs]))

4629808


In [9]:
# Remove words with underscores, since we are going to use them as delimiters in bigrams
docs = [[token for token in doc if '_' not in token] for doc in docs]
print(np.sum([len(doc) for doc in docs]))

4626035


In [10]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Aleksandr.Avdiushenko/nltk_data...


In [13]:
print(lemmatizer.lemmatize('abstracts'),
      lemmatizer.lemmatize('fishes'))

abstract fish


In [14]:
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [15]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)


2024-08-21 10:19:45,717 : INFO : collecting all words and their counts
2024-08-21 10:19:45,717 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-08-21 10:19:48,117 : INFO : collected 1114271 token types (unigram + bigrams) from a corpus of 4626035 words and 1740 sentences
2024-08-21 10:19:48,117 : INFO : merged Phrases<1114271 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>
2024-08-21 10:19:48,117 : INFO : Phrases lifecycle event {'msg': 'built Phrases<1114271 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000> in 2.40s', 'datetime': '2024-08-21T10:19:48.117865', 'gensim': '4.3.3', 'python': '3.11.9 (main, Jul  9 2024, 11:54:13) [Clang 15.0.0 (clang-1500.3.9.4)]', 'platform': 'macOS-14.6.1-arm64-arm-64bit', 'event': 'created'}


In [16]:
for token in bigram[docs[0][:100]]:
    if '_' in token:
        print(token)

abu_mostafa
california_institute
technology_pasadena
ca_abstract
neural_network
boolean_function
can_be
very_low
learning_rule
lower_bound


In [17]:
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document
            docs[idx].append(token)

In [ ]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents
dictionary = Dictionary(docs)

Remove words that are too rare (e.g., typos) and words that are too frequent (e.g., stop words or just common non-topic terms). The `filter_extremes` function removes tokens from the dictionary that appear in less than `no_below` documents or in more than `no_above` fraction of the total number of documents.

In [ ]:
# Remove rare and common tokens
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

Represent all documents in vector form (Bag-of-Words)

In [20]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [21]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 8623
Number of documents: 1740


In [22]:
print(corpus[0][:10])

[(0, 4), (1, 1), (2, 1), (3, 2), (4, 4), (5, 4), (6, 1), (7, 1), (8, 1), (9, 1)]


### Training
Now we are ready to build a topic model for our collection. We will build an online LDA model, implemented in the `gensim` library. We specify the vectorized corpus of texts, the dictionary, and the number of topics (10). We will discuss the remaining parameters later.

In [24]:
start = time()
# Set training parameters.
num_topics = 10
chunksize = 2000  # batch-size
epochs = 5   
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=epochs,
    eval_every=eval_every
)
print('Evaluation time: {}'.format((time()-start) / 60))

2024-08-21 10:27:54,963 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2024-08-21 10:27:54,965 : INFO : using serial LDA version on this node
2024-08-21 10:27:54,971 : INFO : running online (multi-pass) LDA training, 10 topics, 5 passes over the supplied corpus of 1740 documents, updating model once every 1740 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2024-08-21 10:27:54,972 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2024-08-21 10:27:54,972 : INFO : PROGRESS: pass 0, at document #1740/1740
2024-08-21 10:28:00,293 : INFO : optimized alpha [0.09430594, 0.071321055, 0.08636341, 0.07950668, 0.07086547, 0.06970359, 0.084672704, 0.074144214, 0.051429193, 0.08029875]
2024-08-21 10:28:00,297 : INFO : topic #8 (0.051): 0.004*"recognition" + 0.003*"speech" + 0.003*"noise" + 0.003*"class" + 0.0

Evaluation time: 0.2520772457122803


Let's see what we got. We are interested in part of the Phi matrix – the probabilities of words in topics. The NeurIPS collection is entirely dedicated to machine learning. It's difficult to evaluate the topics, though some interpretability can be traced.

In [25]:
for position in range(10):
    row = []
    for topic in range(10):
        row.append(model.show_topic(topic)[position][0].center(11, ' '))
    print(''.join(row))

recognition   hidden     class      layer      memory     action  generalization   image      speech     neuron  
    word      layer      sample      chip      neuron    control   classifier   object     signal      cell   
  gaussian hidden_unitclassification   neuron     matrix     policy   prediction   motion     source    response 
   class       net      distance     net      dynamic      rule      class      visual    sequence   stimulus 
  mixture    control     expert     analog    solution reinforcement   bound    direction    noise      visual  
   hidden    trained     memory      bit        node     optimal    optimal      cell    component    signal  
 likelihood    back      neuron      node     circuit    dynamic      tree      field     language   activity 
   image   architecture    rule      noise       rule   reinforcement_learningtraining_set    view    recurrent    spike   
    node    connection  instance   circuit    hopfield  controllerapproximation   pixel    

In [26]:
top_topics = model.top_topics(corpus)

2024-08-21 10:28:53,602 : INFO : CorpusAccumulator accumulated stats from 1000 documents


In [27]:
top_topics[0]

([(0.024797827, 'image'),
  (0.009840463, 'object'),
  (0.0076078526, 'motion'),
  (0.006597549, 'visual'),
  (0.0063050683, 'direction'),
  (0.0057046246, 'cell'),
  (0.004833315, 'field'),
  (0.004271411, 'view'),
  (0.0041967086, 'pixel'),
  (0.0035704765, 'component'),
  (0.0035410635, 'recognition'),
  (0.0035079652, 'spatial'),
  (0.0032911452, 'filter'),
  (0.003238161, 'vision'),
  (0.0029603366, 'response'),
  (0.0028917522, 'location'),
  (0.0028470703, 'scene'),
  (0.0028185875, 'region'),
  (0.0027665556, 'position'),
  (0.0027341978, 'scale')],
 -0.9543598323604014)

In [28]:
model.inference([corpus[0]])[0]

array([[6.4170547e-02, 4.2514332e-02, 3.2069064e+02, 9.7954498e+01,
        2.4274385e+02, 1.3815321e+02, 5.6009926e-02, 4.4773072e-02,
        3.4585927e-02, 5.3537142e-02]], dtype=float32)

### Perplexity evaluation
We want to assess the model with something more convincing than just looking at topic profiles and document profiles. This is necessary for the possibility of comparing different models, for example, those obtained with different run parameters. Let's learn to measure **perplexity**. The function `model.state.get_lambda` returns the unnormalized $\Phi$ matrix, and `model.inference` estimates the unnormalized $\Theta$ matrix for a list of documents.

We iterate through the collection and calculate perplexity using the formula. The lower the perplexity, the better.

In [33]:
def perplexity(model, corpus):
    corpus_length = 0
    log_likelihood = 0
    topic_profiles = model.state.get_lambda() / np.sum(model.state.get_lambda(), axis=1)[:, np.newaxis]
    for document in corpus:
        gamma, _ = model.inference([document])
        document_profile = gamma / np.sum(gamma)
        for term_id, term_count in document:
            corpus_length += term_count
            term_probability = np.dot(document_profile, topic_profiles[:, term_id])
            log_likelihood += term_count * log(term_probability.item())
    perplexity = np.exp(-log_likelihood / corpus_length)
    return perplexity

In [34]:
print('Perplexity: {}'.format(perplexity(model, corpus)))

Perplexity: 2753.1989393082135


In [ ]:
model_5 = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=5,
    passes=epochs,
    eval_every=eval_every
)
model_20 = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=20,
    passes=epochs,
    eval_every=eval_every
)

In [35]:
print('Perplexity 5: {}'.format(perplexity(model_5, corpus)))
print('Perplexity 20: {}'.format(perplexity(model_20, corpus)))

Perplexity 5: 3120.487941603998
Perplexity 20: 2528.991355165502


### Word2Vec model
Word2Vec is one of the fundamental neural network models of the "pre-transformer" era (2013-2018). The essence of the model is to build a mapping of words into an $N$-dimensional space (embeddings) with certain characteristics. Two words have more similar embeddings the more similar the contexts in which they are used.

In the `gensim` library, two methods for building word2vec are implemented:
  - Skip-grams (SG)
  - Continuous-bag-of-words (CBOW)

## Demo
For the demonstration, let's take a pre-trained model trained on the Google News dataset, containing approximately 3 million English words and phrases.

In [ ]:
# download the model ~1.6GB 
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [40]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key )} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [41]:
vec_king = wv['king']
print(vec_king[:10])

[ 0.12597656  0.02978516  0.00860596  0.13964844 -0.02563477 -0.03613281
  0.11181641 -0.19824219  0.05126953  0.36328125]


Using the model, you can compute the distances between words.

In [42]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


You can also find the most similar words to a given one.

In [43]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

[('SUV', 0.8532192707061768), ('vehicle', 0.8175783753395081), ('pickup_truck', 0.7763689756393433), ('Jeep', 0.7567334175109863), ('Ford_Explorer', 0.7565719485282898)]


In [44]:
vec_example = wv['king'] - wv['man'] + wv['woman']

similars = wv.most_similar(positive=[vec_example])
print(similars)

[('king', 0.8449392318725586), ('queen', 0.7300516366958618), ('monarch', 0.6454660296440125), ('princess', 0.6156251430511475), ('crown_prince', 0.5818676948547363), ('prince', 0.5777117609977722), ('kings', 0.5613663792610168), ('sultan', 0.5376776456832886), ('Queen_Consort', 0.5344247817993164), ('queens', 0.5289887189865112)]


In [45]:
vec_example = wv['programmer'] - wv['man'] + wv['woman'] 

similars = wv.most_similar(positive=[vec_example])
print(similars)

[('programmer', 0.885962188243866), ('programmers', 0.6040860414505005), ('computer_programmer', 0.5623369216918945), ('coder', 0.5616979598999023), ('Programmer', 0.5576066374778748), ('programer', 0.5161396265029907), ('graphic_designer', 0.5139066576957703), ('coders', 0.48765403032302856), ('designer', 0.4822673797607422), ('librarian', 0.4649229943752289)]
